# Generation Evaluation for Redbox RAG  <a class="anchor" id="title"></a>

## Table of Contents <a class="anchor" id="toc"></a>
* [Overview](#one-section)
* [Metrics](#two-section)
    - [Fathfulness]()
    - [Answer Relevancy]()
    - [Hallucination]()
* [Evaluation Dataset](#three-section)
* [Prompt Playground](#four-section)
* [Generate `actual_output` using RAG and evaluation dataset](#five-section)
* [six](#six-section)

## Overview <a class="anchor" id="one-section"></a>

When it comes to optimising the generation part of our RAG system, the only thing that we can modify are the `RAG prompts` that are passed with context to the LLM. Certainly, other things play into the overall generation evaluation score, such as is the retrieved context of high-quality, but the levers to change this are further upstream in the RAG pipeline, and evaluated in Retrieval Evaluation and e2d Evaluation notebooks.

We want to avoid using the /chat/rag endpoint for quick experimentation with `RAG prompts`, as the need to rebuild docker image, start and stop container etc will really slow down development --> changing prompts is very quick to do, so we want quick evaluation of how these prompt changes. 

For this reason, the /chat/rag endpoint function is in this notebook, and prompts can be changed in a single place, followed by much quicker feedback. If the prompt experiments you do look good, there is information at the bottom of this notebook about where you should make the corresponding changes in the core-api to have these changes reflected in the deployed /chat/rag endpoint.

We will evaluate RAG generation using metrics described in the next section.

[Back to top](#title)

## Metrics <a class="anchor" id="two-section"></a>

Start by using 3 DeepEval metrics:
- Faithfulness
- Answer Relevancy **(what are we taking as 'input'? Raw question or refined question?)**
- Hallucination

### Faithfulness

The faithfulness metric measures the quality of your RAG pipeline's generator by evaluating whether the `actual_output` factually aligns with the contents of your `retrieval_context`. `deepeval`'s faithfulness metric is a self-explaining LLM-Eval, meaning it outputs a reason for its metric score.

##### Required Arguments
To use the `FaithfulnessMetric`, you need to provide the following arguments when creating an LLMTestCase:

- `input`
- `actual_output`
- `retrieval_context`

[Back to top](#title)

### Answer Relevancy
The answer relevancy metric measures the quality of your RAG pipeline's generator by evaluating how relevant the actual_output of your LLM application is compared to the provided `input`. `deepeval`'s answer relevancy metric is a self-explaining LLM-Eval, meaning it outputs a reason for its metric score.

##### Required Arguments
To use the AnswerRelevancyMetric, you'll have to provide the following arguments when creating an LLMTestCase:

- `input`
- `actual_output`

[Back to top](#title)

### Hallucination
The hallucination metric determines whether your LLM generates factually correct information by comparing the `actual_output` to the provided `context`.

##### Required Arguments
To use the HallucinationMetric, you'll have to provide the following arguments when creating an LLMTestCase:

- `input`
- `actual_output`
- `retrieval_context`

[Back to top](#title)

## Evaluation Dataset <a class="anchor" id="three-section"></a>

### Load dataset

In [4]:
#TODO: Load evaluation dataset for generation evaluation
from deepeval.dataset import EvaluationDataset

dataset = EvaluationDataset()
dataset.add_test_cases_from_json_file(
    # file_path is the absolute path to you .json file
    file_path="example.json",
    input_key_name="query",
    actual_output_key_name="actual_output",
    expected_output_key_name="expected_output",
    context_key_name="context",
    retrieval_context_key_name="retrieval_context",
)

[Back to top](#title)

## Prompt Playground <a class="anchor" id="four-section"></a>

Set default prompts. **Do an evaluation run through this notebook before your first experiment.** This will set a baseline for you to compare your experiment results against.

After you have done your first run through the notebook, please experiment with these prompts as you see fit.

Things to experiment with:
1. _chat_template
2. CONDENSE_QUESTION_PROMPT
3. _core_redbox_prompt
4. CORE_REDBOX_PROMPT
5. _with_sources_template
6. WITH_SOURCES_PROMPT
7. _stuff_document_template
8. STUFF_DOCUMENT_PROMPT

In [ ]:
from langchain.prompts.prompt import PromptTemplate

#### Refining Question Prompts
As this refining of the question is pre-retrieval, should we **remove it** from this generation evalution notebook?

In [ ]:
_chat_template = """Given the following conversation and a follow up question,
rephrase the follow up question to be a standalone question, in its original
language. include the follow up instructions in the standalone question.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""

In [ ]:
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_chat_template)

#### RAG prompts
These are the prompts that will have most effect on RAG generation

In [ ]:
_core_redbox_prompt = """You are RedBox Copilot. An AI focused on helping UK Civil Servants, Political Advisors and\
Ministers triage and summarise information from a wide variety of sources. You are impartial and\
non-partisan. You are not a replacement for human judgement, but you can help humans\
make more informed decisions. If you are asked a question you cannot answer based on your following instructions, you\
should say so. Be concise and professional in your responses. Respond in markdown format.

=== RULES ===

All responses to Tasks **MUST** be translated into the user's preferred language.\
This is so that the user can understand your responses.\
"""

In [ ]:
# Check where CORE_REDBOX_PROMPT is used in the codebase
CORE_REDBOX_PROMPT = PromptTemplate.from_template(_core_redbox_prompt)

In [ ]:
_with_sources_template = """Given the following extracted parts of a long document and \
a question, create a final answer with Sources at the end.  \
If you don't know the answer, just say that you don't know. Don't try to make \
up an answer.
Be concise in your response and summarise where appropriate. \
At the end of your response add a "Sources:" section with the documents you used. \
DO NOT reference the source documents in your response. Only cite at the end. \
ONLY PUT CITED DOCUMENTS IN THE "Sources:" SECTION AND NO WHERE ELSE IN YOUR RESPONSE. \
IT IS CRUCIAL that citations only happens in the "Sources:" section. \
This format should be <DocX> where X is the document UUID being cited.  \
DO NOT INCLUDE ANY DOCUMENTS IN THE "Sources:" THAT YOU DID NOT USE IN YOUR RESPONSE. \
YOU MUST CITE USING THE <DocX> FORMAT. NO OTHER FORMAT WILL BE ACCEPTED.
Example: "Sources: <DocX> <DocY> <DocZ>"

Use **bold** to highlight the most question relevant parts in your response.
If dealing dealing with lots of data return it in markdown table format.

QUESTION: {question}
=========
{summaries}
=========
FINAL ANSWER:"""

In [ ]:
WITH_SOURCES_PROMPT = PromptTemplate.from_template(_core_redbox_prompt + _with_sources_template)

In [ ]:
_stuff_document_template = "<Doc{parent_doc_uuid}>{page_content}</Doc{parent_doc_uuid}>"

In [ ]:
STUFF_DOCUMENT_PROMPT = PromptTemplate.from_template(_stuff_document_template)

If you find changes to the prompts above improve the generation evaluation scores, please consider making a PR to update the code in main.

All these prompts are locations in [chat.py](../../redbox/llm/prompts/chat.py), except `_core_redbox_prompt` which is located in [core.py](../../redbox/llm/prompts/core.py)

[Back to top](#title)

--------------

## Generate `actual_output` using RAG and evaluation dataset <a class="anchor" id="five-section"></a>

In [ ]:
def rag_chat(chat_request: ChatRequest, user_uuid: Annotated[UUID, Depends(get_user_uuid)]) -> ChatResponse:
    """Get a LLM response to a question history and file

    Args:


    Returns:
        StreamingResponse: a stream of the chain response
    """
    question = chat_request.message_history[-1].text
    previous_history = list(chat_request.message_history[:-1])
    previous_history = ChatPromptTemplate.from_messages(
        (msg.role, msg.text) for msg in previous_history
    ).format_messages()

    docs_with_sources_chain = load_qa_with_sources_chain(
        llm,
        chain_type="stuff",
        prompt=WITH_SOURCES_PROMPT,
        document_prompt=STUFF_DOCUMENT_PROMPT,
        verbose=True,
    )

    condense_question_chain = LLMChain(llm=llm, prompt=CONDENSE_QUESTION_PROMPT)

    standalone_question = condense_question_chain({"question": question, "chat_history": previous_history})["text"]

    docs = vector_store.as_retriever(
        search_kwargs={"filter": {"term": {"creator_user_uuid.keyword": str(user_uuid)}}}
    ).get_relevant_documents(standalone_question)

    result = docs_with_sources_chain(
        {
            "question": standalone_question,
            "input_documents": docs,
        },
    )

    source_documents = [
        SourceDocument(
            page_content=langchain_document.page_content,
            file_uuid=langchain_document.metadata.get("parent_doc_uuid"),
            page_numbers=langchain_document.metadata.get("page_numbers"),
        )
        for langchain_document in result.get("input_documents", [])
    ]
    return ChatResponse(output_text=result["output_text"], source_documents=source_documents)

[Back to top](#title)

-------

### Prepare dataset for evaluation
Process the goldens and convert them into test cases:

In [ ]:
# A hypothetical LLM application example
from chatbot import query
from typing import List
from deepeval.test_case import LLMTestCase
from deepeval.dataset import Golden
...

def convert_goldens_to_test_cases(goldens: List[Golden]) -> List[LLMTestCase]:
    test_cases = []
    for golden in goldens:
        test_case = LLMTestCase(
            input=golden.input,
            # Generate actual output using the 'input' and 'additional_metadata'
            actual_output=query(golden.input, golden.additional_metadata),
            expected_output=golden.expected_output,
            context=golden.context,
        )
        test_cases.append(test_case)
    return test_cases

# Data preprocessing before setting the dataset test cases
dataset.test_cases = convert_goldens_to_test_cases(dataset.goldens)

[Back to top](#title)

In [2]:
from deepeval import evaluate
from deepeval.metrics import AnswerRelevancyMetric
from deepeval.test_case import LLMTestCase

/Users/andy/Library/Caches/pypoetry/virtualenvs/redbox-MiicHf1r-py3.11/lib/python3.11/site-packages/deepeval/__init__.py:42: UserWarning: You are using deepeval version 0.21.36, however version 0.21.42 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(
